In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer, CountVectorizer
from sklearn.model_selection import cross_validate, cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score

import time
import re
import pickle
from string import punctuation
from nltk import pos_tag
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud

from sklearn.svm import LinearSVC, SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier

In [40]:
_ = pd.read_json('../../../data/data.json', lines=True)

_.head(5)

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [41]:
_.shape

(200853, 6)

In [43]:

df2

,category,text
0,GENERAL,2 mass shooting texas last week 1 tvshe left h...
1,SPORTS AND ENTERTAINMENT,smith join diplo nicky jam 2018 world cup offi...
2,SPORTS AND ENTERTAINMENT,hugh grant marries first time age 57the actor ...
3,SPORTS AND ENTERTAINMENT,jim carrey blast castrato adam schiff democrat...
4,SPORTS AND ENTERTAINMENT,julianna margulies us donald trump poop bag pi...
...,...,...
200848,SCIENCE AND TECH,rim ceo thorsten heins significant plan blackb...
200849,SPORTS AND ENTERTAINMENT,maria sharapova stun victoria azarenka austral...
200850,SPORTS AND ENTERTAINMENT,giant patriot jet colt among improbable super ...
200851,SPORTS AND ENTERTAINMENT,aldon smith arrest 49ers linebacker bust duico...


In [20]:
df.loc[_[_.text.isna()].index]

,category,headline,authors,link,short_description,date
77834,COMEDY,Once.,"Marcia Liss, Contributor(Almost) Famous Cartoo...",https://www.huffingtonpost.com/entry/the-ideal...,,2015-10-11
82270,POLITICS,,"Robert Moran, ContributorRobert Moran leads Br...",https://www.huffingtonpost.com/entry/lincoln-2...,,2015-08-22
90134,COMEDY,"No, No, No!","Marcia Liss, Contributor(Almost) Famous Cartoo...",https://www.huffingtonpost.com/entry/no-no-no_...,,2015-05-24
92381,ENTERTAINMENT,WHO Are You Now ?,"Rev. Peter E. Bauer, ContributorUnited Church ...",https://www.huffingtonpost.com/entry/who-are-y...,,2015-04-29
95001,WORLDPOST,,"Natasha Srdoc, ContributorAuthor, Economist, C...",https://www.huffingtonpost.com/entry/us-and-eu...,,2015-03-29
100426,BUSINESS,,"Gary Snyder, ContributorWriter and Media Strat...",https://www.huffingtonpost.com/entry/disney-ce...,,2015-01-25
101479,MEDIA,,"Gary Snyder, ContributorWriter and Media Strat...",https://www.huffingtonpost.com/entry/beverly-h...,,2015-01-13
105318,GREEN,Ένας νεκρός σε συγκρούσεις μεταξύ διαδηλωτών κ...,,https://www.huffingtonpost.com/entry/story_n_6...,,2014-11-30
105740,HEALTHY LIVING,As We Are,"Francesca Milliken, ContributorWrites about be...",https://www.huffingtonpost.com/entry/as-we-are...,,2014-11-25
113471,QUEER VOICES,,"Gary Snyder, ContributorWriter and Media Strat...",https://www.huffingtonpost.com/entry/beverly-h...,,2014-08-28


In [21]:
_[_.text.isna()]

,Unnamed: 0,category,text
77834,77834,SPORTS AND ENTERTAINMENT,NaN
82270,82270,POLITICS,NaN
90134,90134,SPORTS AND ENTERTAINMENT,NaN
92381,92381,SPORTS AND ENTERTAINMENT,NaN
95001,95001,WORLDNEWS,NaN
100426,100426,BUSINESS-MONEY,NaN
101479,101479,MISC,NaN
105318,105318,ENVIRONMENT,NaN
105740,105740,LIFESTYLE AND WELLNESS,NaN
113471,113471,EMPOWERED VOICES,NaN


In [22]:

df

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [23]:
categories = df['category'].value_counts().index

def groupper(grouplist,name):
    for ele in categories:
        if ele in grouplist:
            df.loc[df['category'] == ele, 'category'] = name

In [24]:


groupper( grouplist= ['WELLNESS', 'HEALTHY LIVING','HOME & LIVING','STYLE & BEAUTY' ,'STYLE'] , name =  'LIFESTYLE AND WELLNESS')

groupper( grouplist= [ 'PARENTING', 'PARENTS' ,'EDUCATION' ,'COLLEGE'] , name =  'PARENTING AND EDUCATION')

groupper( grouplist= ['SPORTS','ENTERTAINMENT' , 'COMEDY','WEIRD NEWS','ARTS'] , name =  'SPORTS AND ENTERTAINMENT')

groupper( grouplist= ['TRAVEL', 'ARTS & CULTURE','CULTURE & ARTS','FOOD & DRINK', 'TASTE'] , name =  'TRAVEL-TOURISM & ART-CULTURE')

groupper( grouplist= ['WOMEN','QUEER VOICES', 'LATINO VOICES', 'BLACK VOICES'] , name =  'EMPOWERED VOICES')

groupper( grouplist= ['BUSINESS' ,  'MONEY'] , name =  'BUSINESS-MONEY')

groupper( grouplist= ['THE WORLDPOST' , 'WORLDPOST' , 'WORLD NEWS'] , name =  'WORLDNEWS')

groupper( grouplist= ['ENVIRONMENT' ,'GREEN'] , name =  'ENVIRONMENT')

groupper( grouplist= ['TECH', 'SCIENCE'] , name =  'SCIENCE AND TECH')

groupper( grouplist= ['FIFTY' , 'IMPACT' ,'GOOD NEWS','CRIME'] , name =  'GENERAL')

groupper( grouplist= ['WEDDINGS', 'DIVORCE',  'RELIGION','MEDIA'] , name =  'MISC')



In [25]:


print("We have a total of {} categories now".format(df['category'].nunique()))
df['category'].value_counts()



We have a total of 12 categories now


LIFESTYLE AND WELLNESS          40619
POLITICS                        32739
SPORTS AND ENTERTAINMENT        30296
TRAVEL-TOURISM & ART-CULTURE    20578
EMPOWERED VOICES                15461
PARENTING AND EDUCATION         14780
MISC                            12448
GENERAL                          9663
WORLDNEWS                        8420
BUSINESS-MONEY                   7644
SCIENCE AND TECH                 4260
ENVIRONMENT                      3945
Name: category, dtype: int64

In [26]:
df

,category,headline,authors,link,short_description,date
0,GENERAL,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,SPORTS AND ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,SPORTS AND ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,SPORTS AND ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,SPORTS AND ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26
...,...,...,...,...,...,...
200848,SCIENCE AND TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,"Reuters, Reuters",https://www.huffingtonpost.com/entry/rim-ceo-t...,Verizon Wireless and AT&T are already promotin...,2012-01-28
200849,SPORTS AND ENTERTAINMENT,Maria Sharapova Stunned By Victoria Azarenka I...,,https://www.huffingtonpost.com/entry/maria-sha...,"Afterward, Azarenka, more effusive with the pr...",2012-01-28
200850,SPORTS AND ENTERTAINMENT,"Giants Over Patriots, Jets Over Colts Among M...",,https://www.huffingtonpost.com/entry/super-bow...,"Leading up to Super Bowl XLVI, the most talked...",2012-01-28
200851,SPORTS AND ENTERTAINMENT,Aldon Smith Arrested: 49ers Linebacker Busted ...,,https://www.huffingtonpost.com/entry/aldon-smi...,CORRECTION: An earlier version of this story i...,2012-01-28


In [27]:
df.drop(['authors', 'link', 'date'], axis = 1, inplace = True) 

In [38]:
df.shape

(200853, 2)

In [39]:
df2.shape

(200853, 2)

In [28]:
df['text'] = df['headline'] + df['short_description']
df

,category,headline,short_description,text
0,GENERAL,There Were 2 Mass Shootings In Texas Last Week...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...
1,SPORTS AND ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...
2,SPORTS AND ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...
3,SPORTS AND ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,SPORTS AND ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...
...,...,...,...,...
200848,SCIENCE AND TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,Verizon Wireless and AT&T are already promotin...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
200849,SPORTS AND ENTERTAINMENT,Maria Sharapova Stunned By Victoria Azarenka I...,"Afterward, Azarenka, more effusive with the pr...",Maria Sharapova Stunned By Victoria Azarenka I...
200850,SPORTS AND ENTERTAINMENT,"Giants Over Patriots, Jets Over Colts Among M...","Leading up to Super Bowl XLVI, the most talked...","Giants Over Patriots, Jets Over Colts Among M..."
200851,SPORTS AND ENTERTAINMENT,Aldon Smith Arrested: 49ers Linebacker Busted ...,CORRECTION: An earlier version of this story i...,Aldon Smith Arrested: 49ers Linebacker Busted ...


In [29]:
df.drop(['headline', 'short_description'], axis = 1, inplace = True)
df

,category,text
0,GENERAL,There Were 2 Mass Shootings In Texas Last Week...
1,SPORTS AND ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...
2,SPORTS AND ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 5...
3,SPORTS AND ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...
4,SPORTS AND ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...
...,...,...
200848,SCIENCE AND TECH,RIM CEO Thorsten Heins' 'Significant' Plans Fo...
200849,SPORTS AND ENTERTAINMENT,Maria Sharapova Stunned By Victoria Azarenka I...
200850,SPORTS AND ENTERTAINMENT,"Giants Over Patriots, Jets Over Colts Among M..."
200851,SPORTS AND ENTERTAINMENT,Aldon Smith Arrested: 49ers Linebacker Busted ...


In [30]:


def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN



In [1]:
REMOVE_SPECIAL_CHARACTER = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS = re.compile('[^0-9a-z #+_]')

STOPWORDS = set(stopwords.words('english'))
punctuation = list(punctuation)
STOPWORDS.update(punctuation)

lemmatizer = WordNetLemmatizer()

def clean_text(text):
    # part 1
    text = text.lower() # lowering text
    text = REMOVE_SPECIAL_CHARACTER.sub('', text) # replace REPLACE_BY_SPACE symbols by space in text
    text = BAD_SYMBOLS.sub('', text) # delete symbols which are in BAD_SYMBOLS from text
    
    # part 2
    clean_text = []
    for w in word_tokenize(text):
        if w.lower() not in STOPWORDS:
            pos = pos_tag([w])
            new_w = lemmatizer.lemmatize(w, pos=get_simple_pos(pos[0][1]))
            clean_text.append(new_w)
    text = " ".join(clean_text)
    
    return text

NameError: name 're' is not defined

In [32]:
df['text'] = df['text'].apply(clean_text)

In [33]:
df.to_csv('cleaned_news.csv', index=False, columns=['category', 'text'])

In [37]:

df2.isna().sum()

category     0
text        12
dtype: int64

In [2]:
df2 = pd.read_csv('cleaned_news.csv')
df2


,category,text
0,GENERAL,2 mass shooting texas last week 1 tvshe left h...
1,SPORTS AND ENTERTAINMENT,smith join diplo nicky jam 2018 world cup offi...
2,SPORTS AND ENTERTAINMENT,hugh grant marries first time age 57the actor ...
3,SPORTS AND ENTERTAINMENT,jim carrey blast castrato adam schiff democrat...
4,SPORTS AND ENTERTAINMENT,julianna margulies us donald trump poop bag pi...
...,...,...
200848,SCIENCE AND TECH,rim ceo thorsten heins significant plan blackb...
200849,SPORTS AND ENTERTAINMENT,maria sharapova stun victoria azarenka austral...
200850,SPORTS AND ENTERTAINMENT,giant patriot jet colt among improbable super ...
200851,SPORTS AND ENTERTAINMENT,aldon smith arrest 49ers linebacker bust duico...


In [3]:
df2.isna().sum()

category     0
text        12
dtype: int64

In [4]:
df2.dropna(subset=['text'], inplace=True)

In [5]:
df2.shape

(200841, 2)

In [6]:
X= df2.text
y= df2.category

X_tr, X_test, y_tr, y_test = train_test_split(X, y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_tr, y_tr, random_state=42)

In [7]:
from imblearn.pipeline import make_pipeline
# Creating Models
models = [('Logistic Regression', LogisticRegression(max_iter=1000)),('Random Forest', RandomForestClassifier()),
          ('Linear SVC', LinearSVC()), ('Multinomial NaiveBayes', MultinomialNB()), ('SGD Classifier', SGDClassifier())]

names = []
results = []
model = []
for name, clf in models:
    pipe = make_pipeline(CountVectorizer(max_features=10000, ngram_range=(1, 2)),
                        TfidfTransformer(),
                        SMOTE(),
                        clf )
    # pipe = Pipeline([('vect', CountVectorizer(max_features=30000, ngram_range=(1, 2))),
    #                 ('tfidf', TfidfTransformer()),
    #                 (name, clf),
    #                 ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_val)
    accuracy = accuracy_score(y_pred, y_val)
    
    names.append(name)
    results.append(accuracy)
    model.append(pipe)
    
    msg = "%s: %f" % (name, accuracy)
    print(msg)

Logistic Regression: 0.640475
Random Forest: 0.592729
Linear SVC: 0.621435
Multinomial NaiveBayes: 0.615035
SGD Classifier: 0.608742


In [8]:
# Logistic Regression
filename = 'logreg_model_smote.sav'
pickle.dump(model[0], open(filename, 'wb'))

# Linear SVC
#filename = 'lin_svc_model.sav'
#pickle.dump(model[2], open(filename, 'wb'))

In [9]:
lr_model = pickle.load(open('logreg_model_smote.sav', 'rb'))

In [10]:
text_1 = "Should I disclose my depression struggles during a job interview? Ask HR"
text_2 = "The companies plan to combine Sanderson Farms with Wayne Farms, a Continental Grain subsidiary, to form a new, privately held poultry business. Operations will include poultry processing plants and prepared foods plants across Alabama, Arkansas, Georgia, Louisiana, Mississippi, North Carolina and Texas."
print(lr_model.predict([text_1, text_2]))

['BUSINESS-MONEY' 'POLITICS']
